In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay,precision_score,r2_score, recall_score, f1_score,mean_squared_error
import statsmodels.api as sm

In [3]:
df = pd.read_csv("dispatch_data_updated.csv")

In [32]:
df = df[['ZIPCODE', 'INITIAL_CALL_TYPE', 'POLICEPRECINCT', 'INCIDENT_RESPONSE_SECONDS_QY', 'ZIPCODE_encoded', 'CALL_TYPE_encoded', 'hour', 'day', 'month', 'day_of_week',
         'INCIDENT_DATETIME','Nearest_Traffic_Vol','Distance_m','average_response_time']]

df['response_time_bucket'] = pd.qcut(df['INCIDENT_RESPONSE_SECONDS_QY'], q=3, labels=['Fast', 'Medium', 'Slow'])


In [33]:
X = df[['ZIPCODE_encoded', 'CALL_TYPE_encoded', 'hour', 'day', 'month', 
        'day_of_week', 'POLICEPRECINCT', 'average_response_time','Distance_m','Nearest_Traffic_Vol']]

y = df['response_time_bucket']

In [34]:
cv = StratifiedKFold(n_splits=2,shuffle=True, random_state=42)
tuned_parameters = [{
    'max_depth': [None, 5, 10, 20, 30],
    'min_samples_split': [2,10,20],
    'criterion': ['gini','entropy'],
    'splitter': ['best'],
    'max_features': ['sqrt', 'log2']
}]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=5805)

pre_pruned_model = DecisionTreeClassifier(random_state=42)

pre_pruned_gs = GridSearchCV(pre_pruned_model,param_grid=tuned_parameters, cv = cv,n_jobs=-1,scoring= 'accuracy')
pre_pruned_gs.fit(X_train,y_train)
best_params = grid_search.best_params_
pre_pruned = pre_pruned_gs.best_estimator_
pre_pruned_predictions = pre_pruned.predict(X_test)
pre_pruned_train_predictions = pre_pruned.predict(X_train)

print(f'Test accuracy score {round(accuracy_score(y_test,pre_pruned_predictions),2)}')
print(f'Train accuracy score {round(accuracy_score(y_train,pre_pruned_train_predictions),2)}')


Test accuracy score 0.5
Train accuracy score 0.52


In [45]:
best_params = pre_pruned_gs.best_params_
print(best_params)

{'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 20, 'splitter': 'best'}


In [18]:
bucket_counts = df['response_time_bucket'].value_counts()

# Display the counts
print("Counts of Each Bucket:")
print(bucket_counts)

Counts of Each Bucket:
response_time_bucket
Fast      33112
Medium    33032
Slow      32924
Name: count, dtype: int64


In [43]:
#df.to_csv('dispatch_data_updated.csv', index=False)

#print("DataFrame saved as 'updated_incident_data.csv'")